In [ ]:
import os
import tensorflow

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras import layers
from keras import models
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K 
from keras.applications import ResNet50
from keras.optimizers import SGD

from IPython.display import display, Image
import matplotlib.image as mpimg

import pickle


In [ ]:
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # 3 = INFO, WARNING, and ERROR messages are not printed

train_path = "/kaggle/input/state-farm-distracted-driver-detection/train/"
test_path = "/kaggle/input/state-farm-distracted-driver-detection/test/"
test_path_array = os.listdir(test_path)

driver_dir = "/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv"
driver_imgs = pd.read_csv(driver_dir)
display(driver_imgs.head())

dataset = pd.read_csv("/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv")
dataset.head()

activity_map = {'c0': 'Safe driving', 
                'c1': 'Texting - right', 
                'c2': 'Talking on the phone - right', 
                'c3': 'Texting - left', 
                'c4': 'Talking on the phone - left', 
                'c5': 'Operating the radio', 
                'c6': 'Drinking', 
                'c7': 'Reaching behind', 
                'c8': 'Hair and makeup', 
                'c9': 'Talking to passenger'}

plt.figure(figsize = (12, 20))
image_count = 1
BASE_URL = '/kaggle/input/state-farm-distracted-driver-detection/train/'
for directory in os.listdir(BASE_URL):
    if directory[0] != '.':
        for i, file in enumerate(os.listdir(BASE_URL + directory)):
            if i == 1:
                break
            else:
                fig = plt.subplot(5, 2, image_count)
                image_count += 1
                image = mpimg.imread(BASE_URL + directory + '/' + file)
                plt.imshow(image)
                plt.title(activity_map[directory])

In [ ]:
def create_my_resnet(resnet):    
    model = Sequential()
    model.add(resnet)
    model.add(Flatten())
    model.add(Dense(1024, activation = "relu"))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(Dense(10, activation='softmax'))   
    model.summary()
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['acc'])
    return  model
    
resnet = ResNet50(include_top=False, input_shape=(240, 240, 3))
resnet.summary()

model_resnet = create_my_resnet(resnet)

In [ ]:
# Image augmentation
train_datagen = ImageDataGenerator(rescale = 1.0/255, # Rescales RGB values to between 0 and 1
                                   shear_range = 0.2, # Rotates image randomly
                                   zoom_range = 0.2, # Zooms image randomly
                                   horizontal_flip = False, # No horizontal flip,
                                   brightness_range=[0.2,1.0],
                                   validation_split = 0.2) 

training_set = train_datagen.flow_from_directory('/kaggle/input/state-farm-distracted-driver-detection/train', 
                                                 target_size = (240, 240), 
                                                 batch_size = 32,
                                                 subset = 'training')

validation_set = train_datagen.flow_from_directory('/kaggle/input/state-farm-distracted-driver-detection/train', 
                                                   target_size = (240, 240), 
                                                   batch_size = 32,
                                                   shuffle = False,
                                                   subset = 'validation')

In [ ]:
model_resnet.fit_generator(training_set,
                 steps_per_epoch = 17943/32,
                 epochs = 10,
                 validation_data = validation_set,
                 validation_steps = 4481/32)

filename = 'shinebrightlikeadiamond.sav'
pickle.dump(model_resnet, open(filename, 'wb'))

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1.0/255)

testing_set = test_datagen.flow_from_directory('/kaggle/input/state-farm-distracted-driver-detection/', 
                                                 target_size = (240, 240), 
                                                 shuffle = False,
                                                 classes = ['test'])

In [ ]:
validation_set.reset()
Y_pred = model_resnet.predict_generator(validation_set)
validation_pred = np.argmax(Y_pred, axis=1)

print(classification_report(validation_set.classes,validation_pred))

In [ ]:
testing_set.reset()
predictions = model_resnet.predict_generator(testing_set)

labels = sorted(os.listdir(train_path)[:-1])
img_names = []
for file in testing_set.filenames:
    file = file.split('/')[1]
    img_names+=[file]

img_names = pd.DataFrame(img_names)

pred_df = pd.DataFrame(predictions, columns=labels)
pred_df.insert(0, "img", img_names)


In [ ]:
pred_df.to_csv('shinebrightlikeadiamond.csv')

<a href='shinebrightlikeadiamond.csv'> Download Predictions </a>